Changes:

* Subsetting to genes that are present in all data sources.. array, RNA-seq, and TEMPUS. 10363 genes.
* Subsetting the JIVE gene signatures to those present genes.

Questions: Are there missing genes, in array, that are present in RNA-seq, that would be better to have for TEMPUS?




In [ ]:
#devtools::install_github("gibbsdavidl/robencla", force = F)

In [1]:
library(tidyverse)
library(robencla)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.2.1     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: R6

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: xgboost


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice


Loading required package: plotROC

Loading required package: parallel

Loading required 

In [2]:
arr_f <- read_csv('data/Females_Array_Data_CW.csv')
rna_f <- read_csv('data/Females_RNASeq_Data_CW.csv')
load("data/Females_feature_sel_genelist.rda")

Rows: 140 Columns: 10356
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr     (2): sample, cluster.group
dbl (10354): PDIA2, ZNF195, PIK3C2B, PHB2, SERF2, PLEKHG6, HOXA9, CD2AP, AP3...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 40 Columns: 10356
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr     (2): sample, cluster.group
dbl (10354): PDIA2, ZNF195, PIK3C2B, PHB2, SERF2, PLEKHG6, HOXA9, CD2AP, AP3...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
dim(arr_f)

[1]   140 10356

In [4]:
dim(rna_f)

[1]    40 10356

In [5]:
genelist

[1] "SOX11"    "PDZD2"    "VAV3"     "FRZB"     "PLK2"     "DCX"     
 [7] "GAP43"    "FJX1"     "XAF1"     "INSM1"    "OGFR"     "C1QL1"   
[13] "GNAI2"    "BNIP3"    "VAV3"     "FRZB"     "AQR"      "ARL4C"   
[19] "GAP43"    "FJX1"     "MAFF"     "COTL1"    "OGFR"     "C1QL1"   
[25] "SCN3A"    "RBP1"     "VAV3"     "FRZB"     "TMEM100"  "RBP1"    
[31] "GAP43"    "FJX1"     "DLG5"     "RBP1"     "OGFR"     "C1QL1"   
[37] "IGSF1"    "FA2H"     "VAV3"     "FRZB"     "IGSF1"    "ALK"     
[43] "GAP43"    "FJX1"     "COL9A3"   "FOXJ1"    "OGFR"     "C1QL1"   
[49] "TRAF6"    "STK32B"   "VAV3"     "FRZB"     "ZXDC"     "STK32B"  
[55] "GAP43"    "FJX1"     "STK32B"   "SECISBP2" "OGFR"     "C1QL1"

In [6]:
table(arr_f$cluster.group)


cluster1 cluster2 cluster3 cluster4 cluster5 
      41       24       14       21       40 

In [8]:

# results list
resl <- list()
print('starting')

# for a number of iterations, could be more
for (i in 1:20) {

  print(i)
  idx <- sample(1:nrow(rna_f),size=5,replace = F)
  sampleidx <- rna_f$sample[idx]

  dfa3 <- arr_f[!(arr_f$sample %in% sampleidx), ]
  dfb3 <- rna_f[c(idx), ]

  # our classifier object named Anne.
  anne <- Robencla$new("Anne")

  # xgboost parameters to pass to each sub-classifier in the ensembles
  params <- list(max_depth=12,    # "height" of the tree, 6 is actually default. I think about 12 seems better.  (xgboost parameter)
                 eta=0.2,        # this is the learning rate. smaller values slow it down, more conservative   (xgboost parameter)
                 nrounds=24,     # number of rounds of training, lower numbers less overfitting (potentially)  (xgboost parameter)
                 nthreads=5,     # parallel threads
                 gamma=0.2,        # Minimum loss reduction required to again partition a leaf node. higher number ~ more conservative (xgboost parameter)
                 lambda=1.2,     # L2 regularization term on weights, higher number ~ more conservative (xgboost parameter)
                 alpha=0.2,     # L1 regularization term on weights. higher number ~ more conservative (xgboost parameter)
                 verbose=0,
                 train_perc=0.8,
                 combine_function='median',
                 size=11
  )

  # First we use the training data
  anne$autotrain(data_frame = dfa3,
                 label_name='cluster_group',
                 sample_id = 'sample',
                 data_mode=c('pairs'),  # pairs,sigpairs,quartiles,tertiles,binary,ranks,original #
                 signatures=NULL,
                 pair_list=genelist,  # subset to these genes.
                 params=params)

  # now we apply the classifier to a test set.
  anne$autotest(data_frame = dfb3,#[,colnames(dfa2)], #'data/Males_RNASeq_Data_v3.csv',
                label_name='cluster_group',
                sample_id = 'sample')

  #table(Pred=anne$results()$BestCall, True=anne$test_label)
  resl[[i]] <- data.frame(Pred=anne$results()$BestCall, True=anne$test_label)
}


[1] "starting"


In [ ]:
resdf <- do.call("rbind", resl)
table(resdf$Pred, resdf$True)

In [ ]:

resdf$cluster1 <- as.numeric(resdf$cluster1)
resdf$cluster2 <- as.numeric(resdf$cluster2)
resdf$cluster3 <- as.numeric(resdf$cluster3)
resdf$cluster4 <- as.numeric(resdf$cluster4)
resdf$cluster5 <- as.numeric(resdf$cluster5)
head(resdf)

write.table(as.data.frame(resdf), 'results/female_rnaseq_pairs.csv', sep=',', quote = F, row.names = F)
